In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import mpld3
#mpld3.enable_notebook()

## Comparison of new GC method for fragmented memory
Read in data from results.txt file.  
Has a description, 'Old Verion:', old data, 'New Version:', new data

In [ ]:
with open("../../wasm-of-ocaml/benchmarks/gc/results.txt") as f:
    old_data = []
    new_data = []
    old = True
    for line in f.readlines():
        if line.startswith("New Version"):
            old = False
        if line[0].isdigit():
            iters, mean, std = line.strip().split()
            if old:
                old_data.append((int(iters), float(mean), float(std)))
            else:
                new_data.append((int(iters), float(mean), float(std)))

# reaches limiting behaviour after about 1000 iterations, so truncate data
old_data = old_data[:-9]
new_data = new_data[:-9]

In [ ]:
x = [x[0] for x in old_data]
y1 = np.array([x[1] for x in old_data])
y1err = np.array([x[2] for x in old_data])
y2 = np.array([x[1] for x in new_data])
y2err = np.array([x[2] for x in new_data])

# error bars are 1 std deviation
# Now that all tracing information left out, much closer performance
plt.errorbar(x, y1, yerr=y1err)
plt.errorbar(x, y2, yerr=y2err)

plt.figure()

plt.title("speedup")
plt.plot(x, y1/y2)
plt.ylim(1,np.max(y1/y2)*1.1)
plt.show()

## Benefit of cutoff for repeating GC
alltrees run with size 5, 6, 7, 8 and 9 (shadow stack overflow if tried with 10). Shows that old version and new version without threshold are near identical, but adding that threshold can dramatically reduce execution time. Execution time and memory usage of problem grows exponentially with tree size.

In [ ]:
with open("../../wasm-of-ocaml/benchmarks/gc/alltrees_results.txt") as f:
    old_data1 = []
    new_data1 = []
    new_with_threshold_data = [] # always grow heap if only 1KB (256 words) could be freed
    i = 0
    for line in f.readlines():
        if len(line.strip()) == 0:
            i += 1
        if line[0].isdigit():
            iters, mean, std = line.strip().split()
            if i == 0:
                old_data1.append((int(iters), float(mean), float(std)))
            elif i == 1:
                new_data1.append((int(iters), float(mean), float(std)))
            else:
                new_with_threshold_data.append((int(iters), float(mean), float(std)))

In [ ]:
x = [x[0] for x in old_data1]
y1 = np.array([x[1] for x in old_data1])
y1err = np.array([x[2] for x in old_data1])
y2 = np.array([x[1] for x in new_data1])
y2err = np.array([x[2] for x in new_data1])
y3 = np.array([x[1] for x in new_with_threshold_data])
y3err = np.array([x[2] for x in new_with_threshold_data])

# error bars are 1 std deviation
# Now that all tracing information left out, much closer performance
plt.errorbar(x, y1, yerr=y1err, label="old")
plt.errorbar(x, y2, yerr=y2err, label="new")
plt.errorbar(x, y3, yerr=y3err, label="new with threshold")
plt.legend()
plt.xticks(x)
plt.yscale('log')

plt.figure()
plt.title("speedup")
plt.plot(x, y1/y2, label="no threshold", c="tab:orange")
plt.plot(x, y1/y3, label="threshold", c="g")
plt.legend()
plt.xticks(x)
# mpld3 doesn't seem to work with hlines properly
#plt.hlines(1.0, 5, 9, color='b', linestyle='dotted')
plt.plot([5,9], [1,1], "--")
plt.show()

Tracing data from alltrees 9, showing number of frees each time GC is called. Note the length of the x-axis i.e. the total number of GC calls made.

In [ ]:
frees = []
memgrows = []
with open("../../wasm-of-ocaml/benchmarks/gc/trees9.txt") as f:
    f.readline()
    frees = eval(f.readline())
    memgrows = eval(f.readline())
    
#plt.figure(1,(0.03*len(frees),3))
plt.figure(1,(12,3))

plt.plot(frees, label="frees")
plt.vlines(np.where(memgrows), -300, -50, colors='k')
# don't show the large free of 2x as many objects at the end of each trace
plt.ylim(-200, 2200)
plt.title("Without threshold")

frees = []
memgrows = []
with open("../../wasm-of-ocaml/benchmarks/gc/newtrees9.txt") as f:
    f.readline()
    frees = eval(f.readline())
    memgrows = eval(f.readline())

# mark number of GC calls in optimised approach on the first plot
n = len(frees)
plt.plot([n,n],[-300,2200], "g--")
    
#plt.figure(2,(0.03*len(frees),3))
plt.figure(2,(12,3))

plt.plot(frees, "g", label="frees")
plt.vlines(np.where(memgrows), -300, -50, colors='k')
plt.ylim(-200, 2200)
plt.title("With threshold")
plt.show()